Fase 3: Data Preparation

Objetivo de la fase: El objetivo de la preparación de datos es crear un conjunto de datos limpio y adecuado que permita implementar el modelo de recomendación de libros. Esto incluye la selección de las variables relevantes, el manejo de los datos faltantes, la codificación de variables categóricas, y la transformación de los datos en un formato que pueda ser utilizado en el análisis.

In [1]:
# Imports

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import kaggle
import os

In [2]:
csv_file_path = 'data/goodreads_data.csv'

In [3]:
if not os.path.exists(csv_file_path):
    kaggle.api.dataset_download_files('jishikajohari/best-books-10k-multi-genre-data', path='data/', unzip=True)

In [4]:
df = pd.read_csv(csv_file_path, delimiter=',', on_bad_lines='skip')

In [5]:
# Creamos un nuevo DataFrame para trabajar
df1 = df.copy()

Rellenar datos en la columna 'Description'

In [ ]:
# Ver resumen de valores faltantes
missing_data = df.isnull().sum()
missing_percentage = (missing_data / len(df)) * 100

# Mostrar los resultados
print("Valores faltantes por columna:")
print(missing_data)

print("\nPorcentaje de valores faltantes por columna:")
print(missing_percentage)

df['Description'].fillna('No Description', inplace=True)
missing_data_after = df.isnull().sum()
print("\nValores faltantes después de la imputación:")
print(missing_data_after)

Rellenamos los datos faltantes de Description con "No Description", ya que, no es una columna que utilizaremos.

Eliminar filas en la columna 'Genres'

Hay un total de 960 filas en la columna 'Genre', en donde hay listas, pero no hay información de generos, lo que hemos decidido hacer en este caso es eliminar las filas, ya que, utilizaremos esta columna para la tarea de clasificación en donde tomaremos los datos de 'Genre'.

In [ ]:
# Eliminar filas donde 'Genres' son listas vacías
df = df[~df['Genres'].apply(lambda x: isinstance(x, list) and len(x) == 0)]

In [ ]:
# Obtener el número de filas y columnas
num_rows, num_columns = df.shape
print(f"Número de filas: {num_rows}, Número de columnas: {num_columns}")

Dataset luego de la injección. Nuestro dataset pasa de 10.000 datos a 9.040.

In [ ]:
# Mostrar solo los 20 géneros más comunes
top_genres = df['Main_Genre'].value_counts().nlargest(20).index

# Filtrar el DataFrame para solo los géneros principales
df_top_genres = df[df['Main_Genre'].isin(top_genres)]

# Gráfico con los 20 géneros más comunes
plt.figure(figsize=(10, 6))
sb.countplot(y='Main_Genre', data=df_top_genres, order=df_top_genres['Main_Genre'].value_counts().index)
plt.title('Distribución de los 20 Géneros Principales')
plt.tight_layout()
plt.show()

Eliminar filas en la columna 'Author'

Hay tan solo 28 filas de autores que no se reconocen.

In [ ]:
# Eliminar filas donde el autor es 'Anonymous'
df = df[df['Author'] != 'Anonymous']

# Confirmar que las filas han sido eliminadas
remaining_anonymous_count = df[df['Author'] == 'Anonymous'].shape[0]
print(f"Número de autores 'Anonymous' después de la eliminación: {remaining_anonymous_count}")

In [ ]:
# Obtener el número de filas y columnas
num_rows, num_columns = df.shape
print(f"Número de filas: {num_rows}, Número de columnas: {num_columns}")

Creemos que debemos eliminar las filas en donde no se encuentran 'Authors' y 'Genres', ya que, son columnas primordiales para nuestra tarea de clasificación. En cambio para la columna de 'Description' preferimos rellenarlo con 'No Description', este cambio no deberia influir en nuestro futuros modelos. 

In [ ]:
# Contar los valores en la columna 'Author'
author_counts = df['Author'].value_counts()

# Obtener el conteo de 'Anonymous'
anonymous_count = author_counts.get('Anonymous', 0)
print(f"Número de autores 'Anonymous': {anonymous_count}")

In [ ]:
# Revisar los valores faltantes en el dataset
missing_values = df1.isnull().sum()
print(missing_values)

# Eliminar filas con valores faltantes
df1_clean = df1.dropna()

# O, si es más conveniente, rellenar los valores faltantes
df1_clean = df1.fillna(method='ffill')  # Forward fill

In [ ]:
# Verificar valores faltantes
missing_data = df1.isnull().sum()
missing_percentage = (missing_data / len(df1)) * 100

print("Valores faltantes por columna:")
print(missing_data)
print("\nPorcentaje de valores faltantes por columna:")
print(missing_percentage)

In [ ]:
# Rellenar valores faltantes en 'Description' con 'No Description'
df1['Description'].fillna('No Description', inplace=True)

# Verificar nuevamente los valores faltantes
missing_data_after = df1.isnull().sum()
print("\nValores faltantes después de la imputación:")
print(missing_data_after)

In [ ]:
# Normalizar las variables numéricas
scaler = MinMaxScaler()
df1[['Avg_Rating', 'Num_Ratings']] = scaler.fit_transform(df1[['Avg_Rating', 'Num_Ratings']])

In [ ]:
# Convertir listas en cadenas separadas por comas para las columnas 'Author' y 'Genres'
df1['Genres'] = df1['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df1['Author'] = df1['Author'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Aplicar OneHotEncoding después de convertir listas en cadenas
df1 = pd.get_dummies(df1, columns=['Author', 'Genres'], drop_first=True)

# Mostrar las primeras filas para confirmar el proceso
print(df1.head())

Eliminación de outliers

In [ ]:
# Definir las columnas numéricas en las que queremos detectar y eliminar outliers
numerical_columns = ['Avg_Rating', 'Num_Ratings']  # Ajusta los nombres a tus columnas numéricas

# Función para detectar outliers usando el método del rango intercuartílico (IQR)
def detect_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    
    return outliers

# Detectamos outliers en las variables numéricas
for column in numerical_columns:
    outliers = detect_outliers_iqr(df1, column)
    print(f'Outliers en {column}:')
    print(outliers)

# Función para eliminar outliers en las columnas numéricas
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Filtramos el DataFrame eliminando los outliers
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    return df_filtered

# Para cada columna numérica, eliminamos los outliers correspondientes
for column in numerical_columns:
    df1 = remove_outliers(df1, column)

In [ ]:
# Mostramos el DataFrame sin outliers
print("Data después de eliminar outliers:")
print(df1.describe())

Luego de la eliminación de los outliers quedamos con 9.012 datos.

Para finalizar estas etapa podemos quedarnos con: Existe una correlación entre Avg_Rating y Num_Rating, esto se puede significar a que mientras 
más calificaciones se hacen, el lector suele hacer reseñas. También las calificaciones promedio van entre 3 a 4.5
lo que indica que los libros están bien valorado por los lectores. Hicimos una copia del dataframe para trabajarlo, eliminamos espacio extra de una columna, normalizamos variables numéricas, creamos matriz de variables numericas, agrupamos valores numericos en categorias y Al comienzo teniamos 10.000 datos y luego de la eliminación de outliers quedamos en 9.012.

Aplicamos OneHotEncoder en los datos que vamos a trabajar para la tarea de regresión

In [ ]:
import pandas as pd

# One-Hot Encoding para 'Main_Genre' y 'Author'
X = pd.get_dummies(df[['Num_Ratings', 'Main_Genre', 'Author']], drop_first=True)
y = df['Avg_Rating']

# Ahora X tendrá columnas adicionales para cada categoría en 'Main_Genre' y 'Author'

In [ ]:
print(df1.columns)

In [ ]:
# Imprimir las primeras filas y la forma original del DataFrame
print("DataFrame Original:")
print(df1.head())

# Verificar el tipo de datos en las columnas 'Author' y 'Genres'
print("Tipos de datos:")
print(df1['Author'].apply(type).value_counts())
print(df1['Genres'].apply(type).value_counts())

# Convertir listas en cadenas separadas por comas para 'Genres' y 'Author'
df1['Genres'] = df1['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df1['Author'] = df1['Author'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)

# Imprimir las primeras filas después de la conversión
print("DataFrame después de convertir listas a cadenas:")
print(df1.head())

# Seleccionar las columnas categóricas
categorical_cols = ['Author', 'Genres']

# Aplicar OneHotEncoder y transformar los datos
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')

# Comprobar si las columnas existen
if all(col in df1.columns for col in categorical_cols):
    encoded_data = encoder.fit_transform(df1[categorical_cols])
else:
    print(f"Las columnas {categorical_cols} no están en el DataFrame.")

# Convertir el resultado en un DataFrame y agregar los nombres de las nuevas columnas
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Unir el DataFrame original con el DataFrame de variables codificadas
df_encoded = pd.concat([df1.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)

# Eliminar las columnas categóricas originales si no se necesitan
df_encoded = df_encoded.drop(columns=categorical_cols)

# Mostrar las primeras filas para verificar el resultado
print("DataFrame Codificado:")
print(df_encoded.head())
print("Forma del DataFrame Codificado:", df_encoded.shape)

In [ ]:
# Seleccionar las columnas categóricas
categorical_cols = ['Author', 'Genres']

# Aplicar OneHotEncoder y transformar los datos
encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
encoded_data = encoder.fit_transform(df1[categorical_cols])

# Convertir el resultado en un DataFrame y agregar los nombres de las nuevas columnas
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_cols))

# Unir el DataFrame original con el DataFrame de variables codificadas
df_encoded = pd.concat([df1.reset_index(drop=True), encoded_df], axis=1)

# Eliminar las columnas categóricas originales si no se necesitan
df_encoded = df_encoded.drop(columns=categorical_cols)

# Mostrar las primeras filas para verificar el resultado
print(df_encoded.head())

Codificamos las variables categoricas y normalizamos los datos.

Preparamos datos para el modelo de regresión

In [ ]:
df['Num_Ratings'] = pd.to_numeric(df['Num_Ratings'], errors='coerce')  # Convertir a numérico si es necesario

# Seleccionar las columnas relevantes
features = ['Num_Ratings', 'Genres', 'Author']  # Puedes agregar otras si las consideras útiles
target = 'Avg_Rating'  # La variable objetivo

# Dividir las características (X) y la variable objetivo (y)
X = df[features]
y = df[target]

# Separar variables numéricas y categóricas
numerical_features = ['Num_Ratings']
categorical_features = ['Genres', 'Author']

# Definir transformador para variables numéricas (escalado)
numerical_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())  # Estandarizar los datos
])

# Definir transformador para variables categóricas (OneHotEncoding)
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # Codificación one-hot para las categóricas
])

# Crear un preprocesador que aplica diferentes transformaciones
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar el preprocesamiento al conjunto de entrenamiento
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

print("Datos preparados para el modelo de regresión.")
print("X_train_processed:", X_train_processed.shape)
print("X_test_processed:", X_test_processed.shape)

Normalizamos los datos para que todas las características tengan una escala similar, evitando que características con valores más grandes (por ejemplo, ratings_count) dominen el modelo.
También aplicamos One-Hot Encoding a las variables categóricas (como autores) para convertirlas en variables numéricas, ya que los algoritmos de regresión no pueden manejar datos categóricos directamente.

In [ ]:
# Convertir variables categóricas a variables dummy
X = pd.get_dummies(df1[['Num_Ratings', 'Main_Genre', 'Author']], drop_first=True)
y = df1['Avg_Rating']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características (opcional)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)